In [1]:
import numpy as np
import pickle
from sklearn.linear_model import ElasticNetCV, LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import norm
from data_generation import m_0, g_0, get_data
from dml_algorithm import mm_ate, dml_ate

In [2]:
rng = np.random.default_rng(seed=123)

In [3]:
N = 250
y_data, d_data, x_data = get_data(N, rng)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
x_quad_data = poly_features.fit_transform(x_data)
y_train, y_test, d_train, d_test, x_train, x_test, x_quad_train, x_quad_test = train_test_split(y_data, d_data, x_data, x_quad_data, test_size=0.2, random_state=42)

In [9]:
x_quad_data.shape

(16000, 65)

In [5]:
l1_ratio = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
alphas = [0.01, 0.1, 1, 10]

In [6]:
model_g0, model_g1 = ElasticNetCV(l1_ratio=l1_ratio, n_alphas=10, max_iter=5000, n_jobs=-1), ElasticNetCV(l1_ratio=l1_ratio, n_alphas=10, max_iter=5000, n_jobs=-1)
model_m = LogisticRegressionCV(Cs=10, 
                               l1_ratios=[0, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 1], 
                               penalty='elasticnet',
                               solver='saga',
                               max_iter=1000,
                               random_state=42,
                               scoring='neg_brier_score',
                               n_jobs=-1
                              )

In [7]:
%%time
model_m.fit(x_train, d_train)
print(model_m.C_, model_m.l1_ratio_)
print(model_m.predict_proba(x_test)[:20,1])
print(m_0(x_test[:20]))

[0.35938137] [0]
[0.44383452 0.55541957 0.46842812 0.46767629 0.26710132 0.15612746
 0.3199831  0.38920055 0.28451799 0.15969429 0.20203785 0.37329422
 0.23555628 0.35794091 0.20244939 0.69952223 0.187436   0.26146237
 0.12456931 0.76780944]
[0.46757368 0.68835753 0.51556744 0.45580508 0.34717051 0.22212825
 0.61745043 0.53851286 0.54897398 0.21779908 0.36124516 0.27468535
 0.29395314 0.36922943 0.18776022 0.73825664 0.19868991 0.2104203
 0.06405023 0.77528901]
CPU times: total: 1.64 s
Wall time: 735 ms


In [114]:
"""
%%time
model_m.fit(x_quad_train, d_train)
print(model_m.C_, model_m.l1_ratio_)
print(model_m.predict_proba(x_quad_test)[:20,1])
print(m_0(x_test[:20]))
"""

C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\linear_model\_sag.py:35

[0.1] [0.7]
[0.03693531 0.18404655 0.69069903 0.41196625 0.69500165 0.33635643
 0.56763541 0.40901834 0.23107324 0.76968027 0.60815894 0.08349949
 0.24164982 0.89614622 0.35362007 0.40155253 0.43111714 0.18631722
 0.32110142 0.02323991]
[0.02989884 0.18599423 0.77968065 0.31913693 0.70306652 0.32809337
 0.59935589 0.58673348 0.24429792 0.67730265 0.64347342 0.06659785
 0.33693052 0.90120211 0.24828482 0.53668259 0.53383136 0.15935715
 0.79567006 0.02883646]
CPU times: total: 4min 32s
Wall time: 1min 15s


In logistic regression, we do not include second-order interaction terms due to true model and since convergence issues and not-converged model was still very bad and model without interaction terms was much better!

In [8]:
%%time
model_g0.fit(x_train[d_train==0], y_train[d_train==0])
print(model_g0.alpha_, model_g0.l1_ratio_)
print(model_g0.predict(x_test[:20]))
print(g_0(0, x_test[:20]))

0.025849035195606464 1.0
[ 1.03364465  8.01197109  5.65525088  5.01021965  9.30271151  5.25332556
 -0.29075156  9.0914986   4.67652837  5.97021037  7.43185531  1.59118454
  3.84284257  4.76637962  4.37386203  5.55270384 11.2162964   4.65450948
  8.0948331   4.03303047]
[ 1.96422327  9.38049098  4.81121536  4.13197859  9.07879928  5.48112904
  1.64736247  9.0274919   3.77133351  6.56564005  8.27090455 -0.11310208
  2.83579848  3.93886505  3.37506486  4.75874102 10.68547325  4.37337197
  8.47965583  2.97486236]
CPU times: total: 234 ms
Wall time: 272 ms


In [20]:
%%time
model_g0.fit(x_quad_train[d_train==0], y_train[d_train==0])
print(model_g0.alpha_, model_g0.l1_ratio_)
print(model_g0.predict(x_quad_test[:20]))
print(g_0(0, x_test[:20]))

0.055069696856681956 1.0
[10.11023969  3.5665331   3.68564295  6.94684255  5.07217378  4.03738733
  6.76998776  7.3270528   6.85232092  4.42675164  3.01321268  2.84759681
  4.15156876  6.63511058  5.33835492  5.89884912 11.08195023  4.46686605
  4.39853107  7.75513838]
[11.10481305  4.81084654  3.49232755  7.68548576  5.37828441  4.39214381
  6.87728761  6.59457888  7.76416267  4.57344038  3.44898456  2.45140035
  4.70245763  7.06843659  4.86556282  5.81951578 10.55989342  4.63861388
  4.80569948  7.77599696]
CPU times: total: 453 ms
Wall time: 295 ms


In [23]:
%%time
model_g1.fit(x_train[d_train==1], y_train[d_train==1])
print(model_g1.alpha_, model_g1.l1_ratio_)
print(model_g1.predict(x_test[:20]))
print(g_0(1, x_test[:20]))

0.01 0.1
[ 5.16540176  7.78540839  3.79566213  4.06821275  4.4594141   8.05288531
  1.79014551  5.86656702  8.64134184  8.14282038  3.78477313 -0.57869564
  0.39006149  2.2482847   5.91850482  5.75824314  4.1980805   9.57597934
 11.24850319  3.74846144]
[ 6.24751738  5.98585963  4.83808863  3.04843426  4.21928007 10.01507928
  1.04538262  4.55878404  7.12833787  6.87695721  1.94768289 -2.0836684
 -0.59349964  1.4324121   4.4307337   6.38487366  3.44725359 10.02198399
 11.5393095   3.08063252]
CPU times: total: 125 ms
Wall time: 267 ms


In [21]:
%%time
model_g1.fit(x_quad_train[d_train==1], y_train[d_train==1])
print(model_g1.alpha_, model_g1.l1_ratio_)
print(model_g1.predict(x_quad_test[:20]))
print(g_0(1, x_test[:20]))

0.03398592691481528 1.0
[11.77197604  7.02145023  5.20727674  5.74612724  5.29320599  4.1674984
  8.97225707  4.9102522   7.23205243  3.62729984  5.05898436  3.45240891
  5.98632843  6.60612917  6.13399481  6.68611898 10.37241923  4.42497222
  3.67324962  9.85805104]
[12.12457538  7.6064959   5.14624058  6.14682521  5.44233397  4.80327703
  9.12791178  4.72869834  7.67342315  3.76316409  5.19723771  3.38574428
  6.52481393  6.87520649  6.42372262  6.52067982 10.32296312  4.56048883
  3.78389008  9.39637862]
CPU times: total: 750 ms
Wall time: 336 ms


In [25]:
def dml_ate(y_data, d_data, x_data, x_quad_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05):
    # Generate random partition of data for cross-fitting
    N = len(y_data)
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

    # Compute respective ML estimators and thereupon auxiliary estimators
    theta_0_check_list = []
    if classical:
        reg_check_list, ipw_check_list = [], []
    if inference:
        scores_list = []
    
    for (train_indices, eval_indices) in skf.split(X=x_data, y=d_data):
        y_train, d_train, x_train, x_quad_train = y_data[train_indices], d_data[train_indices], x_data[train_indices], x_quad_data[train_indices] 
        y_eval, d_eval, x_eval, x_quad_eval = y_data[eval_indices], d_data[eval_indices], x_data[eval_indices], x_quad_data[eval_indices] 

        # Estimate outcome regression functions g_0(d)
        g_0_hat = []
        for d in [0, 1]:
            model_g[d].fit(X=x_quad_train[d_train==d], y=y_train[d_train==d])
            g_0_hat.append(model_g[d].predict(x_quad_eval))

        # Estimate propensity score m_0
        model_m.fit(X=x_train, y=d_train)
        m_0_hat = model_m.predict_proba(x_eval)[:,1]
            
        # Compute auxiliary estimator
        scores = g_0_hat[1] - g_0_hat[0] + d_eval*(y_eval-g_0_hat[1])/m_0_hat - (1-d_eval)*(y_eval-g_0_hat[0])/(1-m_0_hat)
        theta_0_check_list.append(np.mean(scores))

        # For variance estimation
        if inference:
            scores_list.append(scores)

        # For regression & IPW estimators
        if classical:
            reg_check_list.append(np.mean(g_0_hat[1] - g_0_hat[0])) 
            ipw_check_list.append(np.mean(d_eval*y_eval/m_0_hat - (1-d_eval)*y_eval/(1-m_0_hat)))     

    # Compute final estimator
    theta_0_hat = np.mean(theta_0_check_list)
    if classical:
        reg_hat, ipw_hat = np.mean(reg_check_list), np.mean(ipw_check_list)

    # Inference: estimate variance and construct confidence interval
    if inference:
        sigma_hat = np.sqrt(np.mean((np.array(scores_list)-theta_0_hat)**2))
        quantile = norm.ppf(1-alpha/2)
        CI = np.array([theta_0_hat-quantile*sigma_hat/np.sqrt(N), theta_0_hat+quantile*sigma_hat/np.sqrt(N)])

    # Return results
    if classical:
        if inference:
            return np.array([theta_0_hat, reg_hat, ipw_hat]), sigma_hat, CI
        else:
            return np.array([theta_0_hat, reg_hat, ipw_hat])
    else:
        if inference:
            return theta_0_hat, sigma_hat, CI
        else:
            return theta_0_hat

In [53]:
%%time
model_g = [model_g0, model_g1]
dml_ate(y_data, d_data, x_data, x_quad_data, model_g, model_m)

CPU times: total: 8.69 s
Wall time: 3.6 s


(array([0.62729263, 0.56263539, 0.25905684]),
 2.760932977028119,
 array([0.45617138, 0.79841388]))